In [2]:
import math
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as si

In [37]:
S0 = 100
K = 98
T = [0,0.25,0.5,0.75,1]
dt = 0.25
r = 0
delta = 0.02
#q1 sigma cst = 23%
sigma = 0.23
mcpath = 1000
n = len(T)
#matrix N*5
np.random.seed( 420 ) 

In [38]:
vol = 0.23
var = vol ** 2
dailyVol = np.sqrt( var / dt )
time=1
dt=0.25
inter = time * dt

In [39]:
S = np.zeros((mcpath,n))

In [40]:
for i in range(0,mcpath):
    t = 0 
    W = 0 
    C=np.ones(5)
    for j in range(0,n):
        if j==0:
            C[j]=100
        else:
            W = np.random.normal(0,np.sqrt(dt))
            C[j]=np.exp((r-delta-0.5*sigma**2)*dt+sigma*W)
        S[i,j] = np.prod(C)

In [41]:
# Compute A for t1 t2 t3 t3
A = np.zeros((mcpath,n-1))
A[:,0] = S[:,1]
A[:,1] = (S[:,1] + S[:,2])/2
A[:,2] = (S[:,1] + S[:,2] + S[:,3])/3
A[:,3] = (S[:,1] + S[:,2] + S[:,3] + S[:,4])/4

In [42]:
#def function to do the intrinsic value
def intr(a):
    res = np.array([])
    for k in range(0,len(a)):
        if a[k] > 0:
            res = np.append(res,a[k])
        else:
            res = np.append(res,0)
    return res

In [108]:
C = np.zeros((mcpath,n))
C[:,n-1] = intr(S[:,n-1]-K)
for i in range(1,n):
    X = np.array([np.ones(mcpath),S[:,n-1-i],S[:,n-1-i]**2,S[:,n-1-i]**3,A[:,n-1-i],A[:,n-1-i]**2,A[:,n-1-i]**3]).T
    Y = np.sum(C,axis = 1) 
    beta = np.dot(np.linalg.inv(np.dot(X.T,X)),np.dot(X.T,Y))
    Y_reg = np.dot(X,beta)
    stop = intr(S[:,n-1-i]-K) > Y_reg
    C[:,n-1-i] = intr(S[:,n-1-i]-K)*stop
    for j in range(0,mcpath):
        if (stop[j] == True):  
            if (C[j,n-i-1] > (C[j,n-i])) and (stop[j] == True):
                C[j,n-i:n] = 0

In [109]:
pd.DataFrame(C) 

,0,1,2,3,4
0,0.0,0.00000,0.000000,0.000000,0.000000
1,0.0,0.00000,15.058820,0.000000,0.000000
2,0.0,0.00000,39.863161,0.000000,0.000000
3,0.0,0.00000,0.000000,0.000000,0.000000
4,0.0,0.00000,0.000000,0.000000,11.674056
...,...,...,...,...,...
995,0.0,0.00000,0.000000,36.257348,0.000000
996,0.0,0.00000,0.000000,0.000000,0.000000
997,0.0,0.00000,0.000000,30.627580,0.000000
998,0.0,12.77337,0.000000,0.000000,0.000000


In [110]:
price = sum(sum(C))/mcpath
price 

13.627884653033096

In [111]:
def bs(S, K, T, r, q, sigma):
    
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * np.exp(-q * T) * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

In [112]:
bs(S0, K, 1,r,delta, sigma)

8.983224650185882